<a href="https://colab.research.google.com/github/junwei2110/FreeCodeCamp_MachineLearning/blob/SMS-text-classifier/FCC_SMS_Challenge_Using_words_instead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.preprocessing import sequence
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-07-10 11:02:45--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2021-07-10 11:02:45 (12.0 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2021-07-10 11:02:46--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
# Read the data
training_raw = open(train_file_path, 'rb').read().decode(encoding='utf-8')
testing_raw = open(test_file_path, 'rb').read().decode(encoding='utf-8') 
print(training_raw[:250])

ham	ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.
ham	you can never do nothing
ham	now u sound like manky scouse boy steve,like! i is travelling on d


In [ ]:
# Create a list with the labels and the corresponding text
import re

training_text = re.split('[\t\n]', training_raw)
testing_text = re.split('[\t\n]', testing_raw)

In [ ]:
# Split into the text and the labels

training_text_data = []
training_text_label = []
testing_text_data = []
testing_text_label = []

for i in range(len(training_text)):
  if (i%2 == 0):
    training_text_label.append(training_text[i])
  else:
    training_text_data.append(training_text[i])

for j in range(len(testing_text)):
  if (j%2 == 0):
    testing_text_label.append(testing_text[j])
  else:
    testing_text_data.append(testing_text[j])

In [ ]:
# Check the len of the text and labels

if len(training_text_data) < len(training_text_label):
  training_text_label.pop(-1)
elif len(training_text_data) > len(training_text_label):
  training_text_data.pop(-1)

if len(testing_text_data) < len(testing_text_label):
  testing_text_label.pop(-1)
elif len(testing_text_data) > len(testing_text_label):
  testing_text_data.pop(-1)

In [ ]:
# Our next objective is to create dictionary of character-num pairs. We need to convert the words into numbers to fit into the model, and decode the numbers back into words for interpretation

# We split each element in the training data into a set to iron out all the unique words and then sort them back into a list
training_unique_list = sorted(set(re.split('[\t\n.!$@#%&*()/?><,:''""+-=_ ]', training_raw)))

# Now we create the dictionary using the awesome enumerate method which converts the characters here into tupples paired with the index of the characters
word2idx = {u:i for i,u in enumerate(training_unique_list)}

print(word2idx)

{'': 0, "'": 1, "''": 2, "''ok''": 3, "'anything'": 4, "'comfort'": 5, "'doctors'": 6, "'heart'": 7, "'help'": 8, "'hw": 9, "'if": 10, "'its": 11, "'ll": 12, "'luxury'": 13, "'maangalyam": 14, "'married'": 15, "'melle": 16, "'need'": 17, "'o": 18, "'rencontre'": 19, "'s": 20, "'simple'": 21, "'taxless'": 22, "'terrorist'": 23, "'that": 24, "'til": 25, "'uptown": 26, "'xam": 27, '[colour': 28, '[in': 29, '[sic]': 30, '^': 31, 'a': 32, 'aa': 33, 'aah': 34, 'aaniye': 35, 'aathi': 36, 'ab': 37, 'abbey': 38, 'abeg': 39, 'abi': 40, 'ability': 41, 'abiola': 42, 'abj': 43, 'able': 44, 'abnormally': 45, 'about': 46, 'aboutas': 47, 'abroad': 48, 'absolutely': 49, 'abt': 50, 'abta': 51, 'aburo': 52, 'abuse': 53, 'abusers': 54, 'ac': 55, 'academic': 56, 'acc': 57, 'accent': 58, 'accept': 59, 'access': 60, 'accessible': 61, 'accident': 62, 'accidentally': 63, 'accommodation': 64, 'accomodate': 65, 'accordin': 66, 'accordingly': 67, 'account': 68, 'accounts': 69, 'accumulation': 70, 'ache': 71, 'ach

In [ ]:
# We will allow maximum 100 words per text message

maxlen = 200

In [ ]:
# Now we create the encoder to convert text to int:

def text_to_int(text):
  list_of_int = []
  for c in text:
    if c in word2idx:
      list_of_int.append(word2idx[c])      
    else:
      list_of_int.append(0)

  list_of_int = np.array(list_of_int)  
  # Now we need to stretch the length to ensure that all the strings evaluated are of same character length
  return sequence.pad_sequences([list_of_int], maxlen)[0] 


# We also create the decoder to convert int back to text:
idx2word = np.array(training_unique_list)
def int_to_text(ints):
  #str = ''
  #for i in range(len(ints)):
  #  str = str + training_unique_list[ints[i]]
  #  str = str + " "
  return " ".join(idx2word[ints])

In [ ]:
# To test the above

print(training_text_data[1])
text_as_int = text_to_int(training_text_data[0])
print(text_as_int)
print(int_to_text(text_as_int))

you can never do nothing
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0   32 2505
 2505 2505 2505    0    0    0 2997 6218 5020 5794    0 6437 4047 3106
 1722 3865    0 6218 4206    0 2505   32 1367    0   32    0  446   32
 1367    0 1367 4701 1722   32 3483    0   32  446 4047 6218 5794    0
 6218    0 5794 2505 4047    0 5020 4047    0 2787    0 1367 4047 3865
 5794    0 3201 2787 3106 1722    0 6218    0 4701 2787 2276 2505 5794
    0 3865 4047 6437    0    0    0    0 2787    0 1367 2787 1367 3865
 5794    0 3106 3865 4047 6437    0   32 3865 6784 5794 2505 2787 3865
 2276    0   32  446 4047 6218 5794    0  838 4047 3483 1722 1367 6784
    0 3865 2787 2276 2505 5794    0  446 6218 5794    0 2787    0 2276
 6218 1722 5020 5020    0 2787 3483    0 6218 4206  

In [ ]:
# The next step will be to convert the training_text_data and testing_text_data into integer form

for i in range(len(training_text_data)):
  training_text_data[i] = text_to_int(training_text_data[i])

for j in range(len(testing_text_data)):
  testing_text_data[j] = text_to_int(testing_text_data[j])


In [ ]:
# Next we convert the labels into integer form as well

labels_dict = {"ham" : 0, "spam": 1}
labels_decoder = {0: "ham", 1: "spam"}

def label_to_int(label):
  return labels_dict[label]

def int_to_label(ints):
  return "".join(labels_decoder[ints])

for i in range(len(training_text_label)):
  training_text_label[i] = label_to_int(training_text_label[i])

for j in range(len(testing_text_label)):
  testing_text_label[j] = label_to_int(testing_text_label[j])

In [ ]:
# Finally time to create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(training_unique_list),256),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation = 'sigmoid')    #Sigmoid ensures output is between 0 and 1
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 256)         1762048   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                36992     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,799,073
Trainable params: 1,799,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])

history = model.fit(np.array(training_text_data), np.array(training_text_label), epochs = 20)

Epoch 1/20
131/131 [==============================] - 4s 18ms/step - loss: 0.2927 - acc: 0.8909
Epoch 2/20
131/131 [==============================] - 2s 17ms/step - loss: 0.1949 - acc: 0.9330
Epoch 3/20
131/131 [==============================] - 2s 17ms/step - loss: 0.1575 - acc: 0.9517
Epoch 4/20
131/131 [==============================] - 2s 17ms/step - loss: 0.1382 - acc: 0.9615
Epoch 5/20
131/131 [==============================] - 2s 16ms/step - loss: 0.1618 - acc: 0.9445
Epoch 6/20
131/131 [==============================] - 2s 17ms/step - loss: 0.1300 - acc: 0.9593
Epoch 7/20
131/131 [==============================] - 2s 16ms/step - loss: 0.1230 - acc: 0.9600
Epoch 8/20
131/131 [==============================] - 2s 16ms/step - loss: 0.1261 - acc: 0.9560
Epoch 9/20
131/131 [==============================] - 2s 16ms/step - loss: 0.0990 - acc: 0.9677
Epoch 10/20
131/131 [==============================] - 2s 17ms/step - loss: 0.0962 - acc: 0.9703
Epoch 11/20
131/131 [==================

In [ ]:
results = model.evaluate(np.array(testing_text_data), np.array(testing_text_label))

print(results)

44/44 [==============================] - 1s 8ms/step - loss: 0.0779 - acc: 0.9792
[0.07789862155914307, 0.9791666865348816]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred_text2int = text_to_int(pred_text)
  pred = np.zeros((1,maxlen))
  pred[0] = pred_text2int
  result = model.predict(pred)

  prediction = list(result[0])
  if result < 0.5:
    prediction.append("ham")
  else:
    prediction.append("spam")

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.0011398428, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True
  list_of_pred = []
  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    list_of_pred.append(prediction)
    if prediction[1] != ans:
      passed = False

  print(list_of_pred)

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

[[0.0010925729, 'ham'], [0.98982555, 'spam'], [0.0034092178, 'ham'], [0.00535548, 'ham'], [0.9214998, 'spam'], [0.0013638643, 'ham'], [0.0032854618, 'ham']]
You haven't passed yet. Keep trying.
